![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP/GenericSVMClassifierApproach.ipynb)

# **GenericSVMClassifierApproach**

This notebook will cover the different parameters and usages of `GenericSVMClassifierApproach`.


**📖 Learning Objectives:**
1. Understand how trains a TensorFlow model for `SVMClassifier` of feature vectors.

2. Become comfortable using the different parameters of the annotator.
**🔗 Helpful Links:**

- Documentation : [GenericSVMClassifierApproach](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericsvmclassifier)

- Python Docs : [GenericSVMClassifierApproach](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/classification/generic_svm_classifier/index.html#module-contents)

- Python Docs : [GenericSVMClassifierApproach](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/nlp/annotators/classification/GenericSVMClassifierApproach.html)

- For extended examples of usage, see [Spark NLP Workshop repository](https://colab.research.google.com/drive/12JQuR7cSUeYfCu418REcqkSmkYWq8Efb#scrollTo=Ltt3uq8y9MKR).


## **📜 Background**

`GenericSVMClassifier` is a derivative of [GenericClassifier](https://colab.research.google.com/drive/1O96phialslGizxSt6a0F7KSy-RUvJtTS?usp=sharing) which implements [SVM (Support Vector Machine) classification](https://en.wikipedia.org/wiki/Support_vector_machine). The input to the model is FeatureVector and the output is category annotations with labels and corresponding confidence scores. The scores are standardized using the logistic function so that they vary between 0 and 1.

## **🎬 Colab Setup**

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.2/265.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.2/676.2 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.8 M

In [ ]:
!pip install -q tensorflow==2.12.0 tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 999.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 32.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.12.0 which is incompatible.


In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Please Upload your John Snow Labs License using the button below


Saving spark_nlp_for_healthcare_spark_ocr_8734_532.json to spark_nlp_for_healthcare_spark_ocr_8734_532.json


In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8734_532.json
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in /root/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8734_532.json
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-5.3.2-py3-none-any.whl to /usr/bin/python3
Installed 1 products:
💊 Spark-Healthcare==5.3.2 installed! ✅ Heal the planet with NLP! 


In [ ]:
import pandas as pd

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8734_532.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.3.2, 💊Spark-Healthcare==5.3.2, running on ⚡ PySpark==3.4.0


In [ ]:
spark

## **🖨️ Input/Output Annotation Types**



- Input: `FEATURE_VECTOR`

- Output: `CATEGORY`

## **🔎 Parameters**

- `batchSize`: (int) Batch size

- `dropout`: (float) Dropout coefficient

- `epochsNumber`: (int) Maximum number of epochs to train

- `featureScaling`: (str) Feature scaling method. Possible values are 'zscore', 'minmax' or empty (no scaling)

- `fixImbalance`: (boolean) Fix the imbalance in the training set by replicating examples of under represented categories

- `labelColumn`: (str) Column with label per each document

- `learningRate`: (float) Learning Rate

- `modelFile`: (str) Location of file of the model used for classification

- `multiClass`: (boolean) If multiClass is set, the model will return all the labels with corresponding scores. By default, multiClass is false.

- `outputLogsPath`: (str) Folder path to save training logs. If no path is specified, the logs won't be stored in disk. The path can be a local file path, a distributed file path (HDFS, DBFS), or a cloud storage (S3).

- `validationSplit`: (float) The proportion of training dataset to be used as validation set.The model will be validated against this dataset on each Epoch and will not be used for training. The value should be between 0.0 and 1.0.

# ✍  Explaining GenericSVMClassifierApproach with Example

## 📑 Sample Data

In [ ]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

In [ ]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg.head()

,col1
0,6460590 NEG Clioquinol intoxication occurring ...
1,"8600337 NEG ""Retinoic acid syndrome"" was preve..."
2,8402502 NEG BACKGROUND: External beam radiatio...
3,"8700794 NEG Although the enuresis ceased, she ..."
4,17662448 NEG A 42-year-old woman had uneventfu...


In [ ]:
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


In [ ]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos.head()

,0,1,2,3,4,5,6,7
0,10030778,Intravenous azithromycin-induced ototoxicity.,ototoxicity,43,54,azithromycin,22,34
1,10048291,"Immobilization, while Paget's bone disease was...",increased calcium-release,960,985,dihydrotachysterol,908,926
2,10048291,Unaccountable severe hypercalcemia in a patien...,hypercalcemia,31,44,dihydrotachysterol,94,112
3,10082597,METHODS: We report two cases of pseudoporphyri...,pseudoporphyria,620,635,naproxen,646,654
4,10082597,METHODS: We report two cases of pseudoporphyri...,pseudoporphyria,620,635,oxaprozin,659,668


In [ ]:
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was...",pos
2,Unaccountable severe hypercalcemia in a patien...,pos
3,METHODS: We report two cases of pseudoporphyri...,pos
4,METHODS: We report two cases of pseudoporphyri...,pos


In [ ]:
ade_df= pd.concat([df_neg, df_pos])
ade_df.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


In [ ]:
ade_df["category"].value_counts()


category
neg    16695
pos     6821
Name: count, dtype: int64

In [ ]:
#We will take 50% of data to make a fast training

spark_df = spark.createDataFrame(ade_df).sample(0.5, 42) # limit the data

trainingData, testData = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 9487
Test Dataset Count: 2369


In [ ]:
spark_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)



In [ ]:
spark_df.head(3)


[Row(text=' Although the enuresis ceased, she developed throbbing headaches, nausea, vomiting, paresthesia, lethargy, fatigue, and altered mental status over the next 7 days.', category='neg'),
 Row(text=' Differences in interpretation by patients and healthcare professionals may cause the observed disparities in seriousness and outcome of reported ADRs.', category='neg'),
 Row(text=' The decedent was also prescribed tramadol, gabapentin, cetirizine, modafinil, carisoprodol, and Xyrem.', category='neg')]

## Preparing input: **FEATURE_VECTOR** for SVM Classifier

***We will use 100d embeddings as input, so text will be vectorized after pipeline below***

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+
|                                                        text|category|                                                    document|                                                       token|                                             word_embeddings|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+
| "I couldn't believe how tired I was without realiz

In [ ]:
# we will take only sentence embedings per sentence

trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| "I couldn't believe how tired I was without realizing it...|     neg|[{sentence_embeddings, 0, 105,  "I couldn't believe how t...|
| 'Spontaneous' hypertensive episodes with monoamine oxida...|     neg|[{sentence_embeddings, 0, 70,  'Spontaneous' hypertensive...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
|                   (ClinicalTrials.gov number, NCT00126724.)|     neg|[{sentence_embeddings, 0, 41,  (ClinicalTrials.gov number...|
| 3,4-Diaminopyridine (3,4-DAP) given alone or combined wi...|     neg|[{sentence_embeddings, 0, 155,  3,4-Diaminopyridine (3,4-...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings.printSchema()


root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [ ]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)


+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[-0.029354565, -0.061089993, 0.11724859, 0.048775017, -0.10324493, -0.0483073, -0.020086473, -0.06853799, 0.00301188...|
|[[0.101560354, -0.005510837, 0.14084685, 0.06470216, -0.11270403, -0.13064885, -0.03348895, -0.10510146, 0.085898675,...|
|[[-0.027627619, 0.06392477, 0.20453192, 0.15634763, -0.09255414, 0.008561155, -0.06016796, -0.14871654, -0.049264766,...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



## **💻Pipeline**

In [ ]:
log_folder="logs"
!mkdir -p $log_folder

### **✅ Graph Creation**

In [ ]:
graph_folder = "graph_folder"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

### ⏰**Training with parameters**

```{Param(parent='GenericSVMClassifierApproach_ad6f332b1585', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='GenericSVMClassifierApproach_ad6f332b1585', name='labelColumn', doc='Column with one label per document'): 'label',
 Param(parent='GenericSVMClassifierApproach_ad6f332b1585', name='batchSize', doc='Size for each batch in the optimization process'): 32,
 Param(parent='GenericSVMClassifierApproach_ad6f332b1585', name='epochsN', doc='Number of epochs for the optimization process'): 10,
 Param(parent='GenericSVMClassifierApproach_ad6f332b1585', name='learningRate', doc='Learning rate for the optimization process'): 0.001,
 Param(parent='GenericSVMClassifierApproach_ad6f332b1585', name='dropout', doc='Dropout at the output of each layer'): 0.05,
 Param(parent='GenericSVMClassifierApproach_ad6f332b1585', name='fixImbalance', doc='A flag indicating whenther to balance the trainig set'): False,
 Param(parent='GenericSVMClassifierApproach_ad6f332b1585', name='featureScaling', doc="Feature scaling method. Possible values are 'zscore', 'minmax' or empty (no scaling)"): '',
 Param(parent='GenericSVMClassifierApproach_ad6f332b1585', name='outputLogsPath', doc='Path to folder where logs will be saved. If no path is specified, no logs are generated'): '',
 Param(parent='GenericSVMClassifierApproach_ad6f332b1585', name='multiClass', doc='Whether to return only the label with the highest confidence score or all labels'): False}
```

We will train a new model with GenericSVMClassifierApproach by assesing each parameters.
Parameters are in two categories:
`labelColumn` , `modelFile`, `outputLogPaths` dont have any effect on model performance or quality. So we will not make any tuning on them.


### ▶ `epochsNumber`:

 How many times will our algorithm see the entire dataset and update its weights.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(2)\
    .setBatchSize(128)\
    .setLearningRate(0.015)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
model = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb


In [ ]:
! ls -t $log_folder | head -n 1
! cat $log_folder/$(ls -t $log_folder | head -n 1)


GenericSVMClassifierApproach_dcf6e91bf3c9.log
Training 2 epochs
Epoch 1/2	0.49s	Loss: 46.12651	ACC: 0.70874304
Epoch 2/2	0.13s	Loss: 42.09118	ACC: 0.72695833
Training completed.


As seen above, 2 epoch takes .36 s and reached accuracy .732

In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.015)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])

In [ ]:
%%time
clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb
CPU times: user 677 ms, sys: 45.5 ms, total: 722 ms
Wall time: 20.4 s


PipelineModel_829964db1e36

In [ ]:
! cat $log_folder/$(ls -t $log_folder | head -n 1)

Training 30 epochs
Epoch 1/30	0.20s	Loss: 45.258003	ACC: 0.7143125
Epoch 2/30	0.14s	Loss: 41.869972	ACC: 0.7353125
Epoch 3/30	0.12s	Loss: 41.243862	ACC: 0.745875
Epoch 4/30	0.10s	Loss: 40.81529	ACC: 0.7535208
Epoch 5/30	0.14s	Loss: 40.20795	ACC: 0.75895834
Epoch 6/30	0.36s	Loss: 40.182648	ACC: 0.75941664
Epoch 7/30	0.11s	Loss: 40.391018	ACC: 0.76034725
Epoch 8/30	0.30s	Loss: 40.051723	ACC: 0.7636806
Epoch 9/30	0.12s	Loss: 40.447147	ACC: 0.76191664
Epoch 10/30	0.11s	Loss: 39.928814	ACC: 0.76472914
Epoch 11/30	0.09s	Loss: 40.19316	ACC: 0.7607639
Epoch 12/30	0.11s	Loss: 40.38734	ACC: 0.760875
Epoch 13/30	0.12s	Loss: 40.449894	ACC: 0.7603055
Epoch 14/30	0.12s	Loss: 40.319057	ACC: 0.7626944
Epoch 15/30	0.14s	Loss: 40.35929	ACC: 0.7654028
Epoch 16/30	0.12s	Loss: 40.284546	ACC: 0.76025
Epoch 17/30	0.11s	Loss: 40.4375	ACC: 0.7619097
Epoch 18/30	0.11s	Loss: 40.69856	ACC: 0.7635347
Epoch 19/30	0.11s	Loss: 40.11449	ACC: 0.7660903
Epoch 20/30	0.11s	Loss: 40.145187	ACC: 0.76686114
Epoch 21/30	0.09s

30 epoch takes 5.69 s and reached accuracy  .763

### ▶ `batchsize`

 how many saples are proccessed in paralel during the learning, it depends on computational power and dataset itself, the smaller batchsize, the longer training time and the smaller memory required

In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(4)\
    .setLearningRate(0.015)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])




In [ ]:
%%time
clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb
CPU times: user 840 ms, sys: 48.1 ms, total: 888 ms
Wall time: 49.1 s


PipelineModel_8c4ac4ce8699

In [ ]:
! cat $log_folder/$(ls -t $log_folder | head -n 1)

Training 30 epochs
Epoch 1/30	1.47s	Loss: 1401.0807	ACC: 0.74511665
Epoch 2/30	1.27s	Loss: 1356.2546	ACC: 0.755059
Epoch 3/30	1.31s	Loss: 1354.2933	ACC: 0.7564292
Epoch 4/30	1.17s	Loss: 1365.1493	ACC: 0.7572372
Epoch 5/30	1.20s	Loss: 1369.7385	ACC: 0.75267005
Epoch 6/30	1.29s	Loss: 1355.844	ACC: 0.7536889
Epoch 7/30	1.31s	Loss: 1348.2036	ACC: 0.7593803
Epoch 8/30	1.18s	Loss: 1366.6951	ACC: 0.7517566
Epoch 9/30	1.19s	Loss: 1390.1079	ACC: 0.74522203
Epoch 10/30	1.11s	Loss: 1360.9191	ACC: 0.7514756
Epoch 11/30	1.13s	Loss: 1362.5068	ACC: 0.7546374
Epoch 12/30	1.14s	Loss: 1356.0885	ACC: 0.7538645
Epoch 13/30	1.13s	Loss: 1373.2179	ACC: 0.75045675
Epoch 14/30	1.14s	Loss: 1358.6975	ACC: 0.756113
Epoch 15/30	1.12s	Loss: 1377.9441	ACC: 0.7519674
Epoch 16/30	1.15s	Loss: 1354.3779	ACC: 0.7524241
Epoch 17/30	1.19s	Loss: 1374.7322	ACC: 0.75446177
Epoch 18/30	1.28s	Loss: 1362.9	ACC: 0.75379425
Epoch 19/30	1.31s	Loss: 1378.1489	ACC: 0.7496487
Epoch 20/30	1.12s	Loss: 1361.1206	ACC: 0.7549536
Epoch 21/3



*   in last two run, we set batch size to 128 and 4.
*   training time increased from 4.86 to 37.9 s with same epoch number




In [ ]:
pred_df = model.transform(testData_with_embeddings)


In [ ]:
pred_df.printSchema()
pred_df.select(pred_df.prediction).show(5, truncate=False)
pred_df.select(pred_df.category, pred_df.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- feature_vector: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |

### ▶ `learningrate`

`The learning rate is a hyperparameter in deep learning and many other machine learning algorithms that determines the step size at which the model's weights are updated during training. It is one of the most critical hyperparameters to tune because it has a profound impact on the training process and the final performance of the model.`

In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])




In [ ]:
%%time
model = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb
CPU times: user 611 ms, sys: 25.8 ms, total: 636 ms
Wall time: 16.1 s


In [ ]:
! cat $log_folder/$(ls -t $log_folder | head -n 1)

Training 30 epochs
Epoch 1/30	0.22s	Loss: 53.80681	ACC: 0.70795834
Epoch 2/30	0.14s	Loss: 46.146893	ACC: 0.7132778
Epoch 3/30	0.14s	Loss: 45.21372	ACC: 0.7118125
Epoch 4/30	0.12s	Loss: 44.256702	ACC: 0.7128055
Epoch 5/30	0.10s	Loss: 44.116657	ACC: 0.71086806
Epoch 6/30	0.11s	Loss: 44.161095	ACC: 0.7085139
Epoch 7/30	0.07s	Loss: 43.49274	ACC: 0.71086806
Epoch 8/30	0.07s	Loss: 43.02876	ACC: 0.71206945
Epoch 9/30	0.08s	Loss: 43.11109	ACC: 0.7117014
Epoch 10/30	0.08s	Loss: 42.747177	ACC: 0.71425694
Epoch 11/30	0.08s	Loss: 42.690353	ACC: 0.7138403
Epoch 12/30	0.08s	Loss: 42.341972	ACC: 0.7176458
Epoch 13/30	0.10s	Loss: 42.429787	ACC: 0.71628475
Epoch 14/30	0.10s	Loss: 42.189724	ACC: 0.72050697
Epoch 15/30	0.10s	Loss: 42.013744	ACC: 0.72139585
Epoch 16/30	0.11s	Loss: 41.993114	ACC: 0.72129166
Epoch 17/30	0.10s	Loss: 42.24681	ACC: 0.7205972
Epoch 18/30	0.10s	Loss: 41.99428	ACC: 0.72216666
Epoch 19/30	0.09s	Loss: 41.60471	ACC: 0.72921526
Epoch 20/30	0.09s	Loss: 41.75936	ACC: 0.7269653
Epoch 21

30 epochs, 128 batch size and 0.001 learning  ▶ 4.04 s trainin time and .729 accuracy

In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.00001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])

In [ ]:
%%time
model = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb
CPU times: user 596 ms, sys: 24.6 ms, total: 621 ms
Wall time: 15.8 s


In [ ]:
! ls -t $log_folder | head -n 1
! cat $log_folder/$(ls -t $log_folder | head -n 1)

GenericSVMClassifierApproach_21237e5ff137.log
Training 30 epochs
Epoch 1/30	0.18s	Loss: 98.5718	ACC: 0.29268056
Epoch 2/30	0.09s	Loss: 97.82649	ACC: 0.29184029
Epoch 3/30	0.09s	Loss: 97.18733	ACC: 0.29142362
Epoch 4/30	0.07s	Loss: 96.627174	ACC: 0.28927776
Epoch 5/30	0.07s	Loss: 95.64738	ACC: 0.29184029
Epoch 6/30	0.08s	Loss: 95.00515	ACC: 0.29251388
Epoch 7/30	0.08s	Loss: 94.485344	ACC: 0.29229861
Epoch 8/30	0.09s	Loss: 93.687775	ACC: 0.29376388
Epoch 9/30	0.09s	Loss: 93.12959	ACC: 0.2932361
Epoch 10/30	0.07s	Loss: 92.44218	ACC: 0.29559028
Epoch 11/30	0.10s	Loss: 91.72659	ACC: 0.2933889
Epoch 12/30	0.10s	Loss: 90.883934	ACC: 0.30090973
Epoch 13/30	0.10s	Loss: 90.342064	ACC: 0.30074304
Epoch 14/30	0.09s	Loss: 89.46234	ACC: 0.30454862
Epoch 15/30	0.07s	Loss: 88.853096	ACC: 0.30386806
Epoch 16/30	0.09s	Loss: 88.1213	ACC: 0.30871528
Epoch 17/30	0.08s	Loss: 87.49541	ACC: 0.31502777
Epoch 18/30	0.08s	Loss: 86.75138	ACC: 0.31814584
Epoch 19/30	0.08s	Loss: 86.22007	ACC: 0.32350695
Epoch 20/30

30 epochs, 128 batch size and 0.001 learning  ▶ 5.82 s trainin time and .745 accuracy

30 epochs, 128 batch size and 0.00001 learning  ▶ 5.71 s trainin time and .68 accuracy

because we are using small amount of data for training purposes, there is small differences in metrics, but normally, learning rate is very crucial.
in this demo, we see the accuracy decreased to sharply, smaller learning rate needs more epoch to reach optimums

### ▶`featureScaling`

In [ ]:
rdf = pd.DataFrame()

In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])
model = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb


In [ ]:
result_ = model.transform(trainingData_with_embeddings)

In [ ]:
result_.select(result_.text, result_.prediction.metadata).show(5, truncate=50)

+--------------------------------------------------+----------------------------+
|                                              text|         prediction.metadata|
+--------------------------------------------------+----------------------------+
| "I couldn't believe how tired I was without re...| [{confidence -> 0.8141968}]|
| 'Spontaneous' hypertensive episodes with monoa...|[{confidence -> 0.68224895}]|
| (2) Rehabilitation of a 29-year-old man with a...| [{confidence -> 0.7901555}]|
| 1). There was no involvement of other intertri...|[{confidence -> 0.84604967}]|
| 1. Nine patients in whom acute non-lymphoblast...| [{confidence -> 0.7506386}]|
+--------------------------------------------------+----------------------------+
only showing top 5 rows



In [ ]:
result_.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- feature_vector: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |

In [ ]:
rdf_ = result_.select(result_.text,result_.category, result_.prediction.result[0],result_.prediction.metadata[0]).toPandas()
rdf_.columns  = ['text','category', 'predicted category', 'conf']
rdf_['confidence'] = rdf_.conf.apply(lambda conf:float(conf['confidence']))
rdf_ = rdf_[['text','category', 'predicted category', 'confidence']]
rdf = rdf_[['text','category', 'predicted category']]
rdf['conf_'] = rdf_['confidence']
rdf.head()

,text,category,predicted category,conf_
0,"""I couldn't believe how tired I was without r...",neg,neg,0.814197
1,'Spontaneous' hypertensive episodes with mono...,neg,neg,0.682249
2,(2) Rehabilitation of a 29-year-old man with ...,neg,neg,0.790156
3,1). There was no involvement of other intertr...,neg,neg,0.846050
4,1. Nine patients in whom acute non-lymphoblas...,neg,neg,0.750639


In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setFeatureScaling('zscore')\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])
model = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb


In [ ]:
result_zscore = model.transform(trainingData_with_embeddings)

In [ ]:
rdf_zscore = result_zscore.select(result_zscore.text,result_zscore.category, result_zscore.prediction.result[0],result_zscore.prediction.metadata[0]).toPandas()
rdf_zscore.columns  = ['text','category', 'predicted category', 'conf']
rdf_zscore['confidence'] = rdf_zscore.conf.apply(lambda conf:float(conf['confidence']))
rdf['conf_zscore'] = rdf_zscore['confidence']

rdf.head()

,text,category,predicted category,conf_,conf_zscore
0,"""I couldn't believe how tired I was without r...",neg,neg,0.814197,0.806100
1,'Spontaneous' hypertensive episodes with mono...,neg,neg,0.682249,0.648858
2,(2) Rehabilitation of a 29-year-old man with ...,neg,neg,0.790156,0.693513
3,1). There was no involvement of other intertr...,neg,neg,0.846050,0.806925
4,1. Nine patients in whom acute non-lymphoblas...,neg,neg,0.750639,0.672616


In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setFeatureScaling('minmax')\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])
model = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb


In [ ]:
result_minmax = model.transform(trainingData_with_embeddings)


In [ ]:
rdf_minmax = result_minmax.select(result_minmax.text,result_minmax.category, result_minmax.prediction.result[0],result_minmax.prediction.metadata[0]).toPandas()
rdf_minmax.columns  = ['text','category', 'predicted category', 'conf']
rdf_minmax['confidence'] = rdf_minmax.conf.apply(lambda conf:float(conf['confidence']))

rdf['conf_minmax'] = rdf_minmax['confidence']
rdf.head()

,text,category,predicted category,conf_,conf_zscore,conf_minmax
0,"""I couldn't believe how tired I was without r...",neg,neg,0.814197,0.806100,0.844715
1,'Spontaneous' hypertensive episodes with mono...,neg,neg,0.682249,0.648858,0.835000
2,(2) Rehabilitation of a 29-year-old man with ...,neg,neg,0.790156,0.693513,0.834357
3,1). There was no involvement of other intertr...,neg,neg,0.846050,0.806925,0.851563
4,1. Nine patients in whom acute non-lymphoblas...,neg,neg,0.750639,0.672616,0.832595


**As shown above, three different featureScaling option generate different confidence, and the best one may be chosen according to problem/dataset**

### ▶`ValidationSplit`

 when `.setValidationSplit(0.1)` set, logs are also having validation scores, by doing so, we can see our model quality during the training process by comparing validatin accuracy score and training accuracy score.

In [ ]:
! ls -t $log_folder | head -n 1
! cat $log_folder/$(ls -t $log_folder | head -n 1)

GenericSVMClassifierApproach_199b52743bd0.log
Training 30 epochs
Epoch 1/30	0.15s	Loss: 48.391003	ACC: 0.6909696	Validation ACC: 0.7291886
Epoch 2/30	0.07s	Loss: 45.898594	ACC: 0.7076026	Validation ACC: 0.7291886
Epoch 3/30	0.07s	Loss: 45.121433	ACC: 0.7077244	Validation ACC: 0.7291886
Epoch 4/30	0.07s	Loss: 44.188213	ACC: 0.70691335	Validation ACC: 0.7291886
Epoch 5/30	0.07s	Loss: 43.820538	ACC: 0.7091418	Validation ACC: 0.7291886
Epoch 6/30	0.07s	Loss: 43.02599	ACC: 0.70824003	Validation ACC: 0.7291886
Epoch 7/30	0.07s	Loss: 42.61813	ACC: 0.70911074	Validation ACC: 0.7291886
Epoch 8/30	0.07s	Loss: 42.122402	ACC: 0.7079317	Validation ACC: 0.7291886
Epoch 9/30	0.08s	Loss: 41.51672	ACC: 0.7094864	Validation ACC: 0.7291886
Epoch 10/30	0.07s	Loss: 41.477657	ACC: 0.7093387	Validation ACC: 0.7291886
Epoch 11/30	0.07s	Loss: 40.975697	ACC: 0.70947343	Validation ACC: 0.7291886
Epoch 12/30	0.07s	Loss: 40.551167	ACC: 0.7087609	Validation ACC: 0.7291886
Epoch 13/30	0.07s	Loss: 40.620594	ACC: 0.70

### ▶ `multiClass`

Whether to return only the label with the highest confidence score or all labels

In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setFeatureScaling('minmax')\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    .setValidationSplit(0.1)\
    .setMultiClass(True)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])
model = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb


In [ ]:
preds_df = model.transform(testData_with_embeddings)

In [ ]:
preds_df.select('prediction').show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------+
|prediction                                                                                                    |
+--------------------------------------------------------------------------------------------------------------+
|[{category, 0, 0, neg, {confidence -> 0.8362459}, []}, {category, 0, 0, pos, {confidence -> 0.62771255}, []}] |
|[{category, 0, 0, neg, {confidence -> 0.83818376}, []}, {category, 0, 0, pos, {confidence -> 0.67019814}, []}]|
|[{category, 0, 0, neg, {confidence -> 0.8433539}, []}, {category, 0, 0, pos, {confidence -> 0.63883567}, []}] |
|[{category, 0, 0, neg, {confidence -> 0.86410296}, []}, {category, 0, 0, pos, {confidence -> 0.70451}, []}]   |
|[{category, 0, 0, neg, {confidence -> 0.82108605}, []}, {category, 0, 0, pos, {confidence -> 0.6275029}, []}] |
+-----------------------------------------------------------------------------------------------

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.76      0.97      0.85      1699
         pos       0.75      0.23      0.35       670

    accuracy                           0.76      2369
   macro avg       0.75      0.60      0.60      2369
weighted avg       0.76      0.76      0.71      2369



### ▶ `setFixImbalance`

default False
when set True:
Algorithm duplicates the minor classes randomly

In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setFeatureScaling('minmax')\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(False)\
    .setMultiClass(False)
    # .setValidationSplit(0.1)\

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])
model = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb


In [ ]:
preds_df = model.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.76      0.97      0.85      1699
         pos       0.75      0.23      0.35       670

    accuracy                           0.76      2369
   macro avg       0.75      0.60      0.60      2369
weighted avg       0.76      0.76      0.71      2369



In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setFeatureScaling('minmax')\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    .setMultiClass(False)
    # .setValidationSplit(0.1)\

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])

model = clf_Pipeline.fit(trainingData_with_embeddings)

preds_df = model.transform(testData_with_embeddings)
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb
              precision    recall  f1-score   support

         neg       0.76      0.97      0.85      1699
         pos       0.75      0.23      0.35       670

    accuracy                           0.76      2369
   macro avg       0.75      0.60      0.60      2369
weighted avg       0.76      0.76      0.71      2369



### ▶ `setDropout`

Dropout is a regularization technique commonly used in deep learning algorithms, especially in neural networks, to prevent overfitting and improve the generalization ability of the model. The "dropout ratio" is a key hyperparameter associated with dropout, and it refers to the probability that a particular neuron or unit in a neural network layer will be "dropped out" or temporarily removed during each training iteration

**`default:0.05`**

In [ ]:
# letx compare 0.015 and 0.00

In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setFeatureScaling('minmax')\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.015)\
    .setFixImbalance(True)\
    .setMultiClass(False)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])
model = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb


In [ ]:
preds_df = model.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.76      0.97      0.85      1699
         pos       0.75      0.23      0.35       670

    accuracy                           0.76      2369
   macro avg       0.75      0.60      0.60      2369
weighted avg       0.76      0.76      0.71      2369



In [ ]:
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(128)\
    .setLearningRate(0.001)\
    .setFeatureScaling('minmax')\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.00)\
    .setFixImbalance(True)\
    .setMultiClass(False)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])
model = clf_Pipeline.fit(trainingData_with_embeddings)


TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: graph_folder
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to graph_folder/svm_graph.pb


In [ ]:
preds_df = model.transform(testData_with_embeddings)
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.76      0.97      0.85      1699
         pos       0.75      0.23      0.35       670

    accuracy                           0.76      2369
   macro avg       0.75      0.60      0.60      2369
weighted avg       0.76      0.76      0.71      2369



***f1-score for pos changed dramatically***